<a href="https://colab.research.google.com/github/federicotarozzi/Corso_AnalistaProgrammatoreDatiGeoSpaziali/blob/main/1_download_climateData_cds_climate_Copernicus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download of climate data by Copernicus data service
This notebook will show a way to extract climate data directly from CDS free of charge through an API. These types of downloads take a long time, so this will be a small example with a few coordinates and only one month.
Hourly ERA5 data of precipitation and temperature will be extracted.

#### Import the library we need

In [3]:
%pip install -q cdsapi

import cdsapi
import xarray as xr
import numpy as np
import pandas as pd
from itertools import product
import calendar
import os

  Preparing metadata (setup.py) ... done


#### Define API keys and identification

In [6]:
# Define your CDS API key
# api_kay is composed by UID: API kay (you can find it on the personal page on CDS)
udi = '322855'
api_key = '3fdd1aca-4ad4-45dc-af43-604d19f82ef0'

#api_kay composed by UID: API kay (you can find it on the personal page on CDS)

# Initialize the CDS API client
c = cdsapi.Client(udi, api_key) # libreria per copernicus per estrarre di dati

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
coords = pd.read_csv('Data/climate_data/sample_coords.csv')
coords

,Province,latitude,longitude
0,Ravenna,44.370647,11.995786
1,Foggia,41.539188,15.560359


#### Run the code for retrieve the data from CDA

In [ ]:
# Cartella di destinazione per i file NetCDF
output_folder = 'Data/climate_data/NetCDF'
# Crea la cartella se non esiste già
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

climate_data = []

# Create an instance of the CDS API client
c = cdsapi.Client()

# Iterate over each coordinate in the "coords" dataset
for index, row in coords.iterrows():
    # Define the period
    start_year = 2021
    end_year = 2021

    years = [str(year) for year in range(start_year, end_year + 1)]
    months = ['04']  # Example Aprile
    # months = ['04', '03', '05', '06', '07', '08', '09']  # For all the growing season of tomato crop

    # Iterate over each year and month
    for year, month in product(years, months):
        num_days_in_month = calendar.monthrange(int(year), int(month))[1]
        days = [str(day).zfill(2) for day in range(1, num_days_in_month + 1)]

        # Iterate over each day
        for day in days:
            hours = [str(hour).zfill(2) for hour in range(24)]  # Hours from 00 to 23

            # Empty DataFrame
            prova = pd.DataFrame()

            # Iterate over each hour
            for hour in hours:
                # Define the request parameters
                request = {
                    'product_type': 'reanalysis',
                    'format': 'netcdf',
                    'variable': ['2m_temperature', 'total_precipitation'],
                    'year': year,
                    'month': month,
                    'day': day,
                    'time': [f'{hour}:00'],  # Get data for the specified hour
                    'area':  [row['latitude'], row['longitude'], row['latitude'], row['longitude']],  # Specify the bounding box (use the same lat/lon for a single point)
                }

                # Generate a unique filename for each request
                filename = os.path.join(output_folder, f'download_{year}_{month}_{day}_{hour}.nc')

                # Request the data from CDS API
                c.retrieve('reanalysis-era5-single-levels', request, filename)

                # Open the downloaded NetCDF file
                data = xr.open_dataset(filename)

                # Print the dataset to check its content
                print(data)

                # Extract temperature data for the hour
                temperature = data['t2m'].sel(time=f'{year}-{month}-{day}T{hour}:00', method='nearest').values
                # Extract precipitation data for the hour
                precipitation = data['tp'].sel(time=f'{year}-{month}-{day}T{hour}:00', method='nearest').values
                # Create a DataFrame for the hour
                hour_df = pd.DataFrame({'2m_temperature': temperature.flatten(), 'total_precipitation': precipitation.flatten()})
                # Add columns for year, month, day, and time
                hour_df['Year'] = year
                hour_df['Month'] = month
                hour_df['Day'] = day
                hour_df['Time'] = f'{hour}:00'
                hour_df['Province'] = row['Province']
                # Concatenate the DataFrame for the hour with the overall DataFrame
                prova = pd.concat([prova, hour_df], ignore_index=True)

                print('done')

                climate_data.append(prova)

final_df = pd.concat(climate_data, ignore_index=True)
final_df


2024-06-26 09:55:47,433 INFO Welcome to the CDS
2024-06-26 09:55:47,435 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-06-26 09:55:47,513 INFO Request is completed
2024-06-26 09:55:47,514 INFO Downloading https://download-0000-clone.copernicus-climate.eu/cache-compute-0000/cache/data9/adaptor.mars.internal-1719319719.4924064-2419-18-275740b0-1608-4028-9c0a-3489cea3525d.nc to Data/climate_data/NetCDF/download_2021_04_01_00.nc (1.3K)
2024-06-26 09:55:47,559 INFO Download rate 29.3K/s
2024-06-26 09:55:47,601 INFO Welcome to the CDS
2024-06-26 09:55:47,601 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-06-26 09:55:47,700 INFO Downloading https://download-0004-clone.copernicus-climate.eu/cache-compute-0004/cache/data4/adaptor.mars.internal-1719319724.424085-17001-17-e5d7c8fa-c667-42c4-9d5d-5627ccba9373.nc to Data/climate_data/NetCDF/download_2021_04_01_01.nc (1.3K)
2

<xarray.Dataset>
Dimensions:    (latitude: 1, longitude: 1, time: 1)
Coordinates:
  * longitude  (longitude) float32 12.0
  * latitude   (latitude) float32 44.37
  * time       (time) datetime64[ns] 2021-04-01
Data variables:
    t2m        (time, latitude, longitude) float32 ...
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-25 12:48:39 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...
done
<xarray.Dataset>
Dimensions:    (latitude: 1, longitude: 1, time: 1)
Coordinates:
  * longitude  (longitude) float32 12.0
  * latitude   (latitude) float32 44.37
  * time       (time) datetime64[ns] 2021-04-01T01:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-25 12:48:44 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...
done


2024-06-26 09:55:47,825 INFO Downloading https://download-0008-clone.copernicus-climate.eu/cache-compute-0008/cache/data1/adaptor.mars.internal-1719319726.5503538-13630-13-0e221828-0e1b-4dd2-a3e0-dac9a9d79eb2.nc to Data/climate_data/NetCDF/download_2021_04_01_02.nc (1.3K)
2024-06-26 09:55:47,876 INFO Download rate 25.9K/s
2024-06-26 09:55:47,913 INFO Welcome to the CDS
2024-06-26 09:55:47,914 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-06-26 09:55:47,996 INFO Downloading https://download-0016.copernicus-climate.eu/cache-compute-0016/cache/data5/adaptor.mars.internal-1719319730.1281104-8948-5-5b1dce5b-f14a-4406-aaae-ba27e6a25d6f.nc to Data/climate_data/NetCDF/download_2021_04_01_03.nc (1.3K)
2024-06-26 09:55:48,040 INFO Download rate 30K/s
2024-06-26 09:55:48,077 INFO Welcome to the CDS
2024-06-26 09:55:48,079 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


<xarray.Dataset>
Dimensions:    (latitude: 1, longitude: 1, time: 1)
Coordinates:
  * longitude  (longitude) float32 12.0
  * latitude   (latitude) float32 44.37
  * time       (time) datetime64[ns] 2021-04-01T02:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-25 12:48:46 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...
done
<xarray.Dataset>
Dimensions:    (latitude: 1, longitude: 1, time: 1)
Coordinates:
  * longitude  (longitude) float32 12.0
  * latitude   (latitude) float32 44.37
  * time       (time) datetime64[ns] 2021-04-01T03:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-25 12:48:50 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...
done


2024-06-26 09:55:48,154 INFO Downloading https://download-0008-clone.copernicus-climate.eu/cache-compute-0008/cache/data1/adaptor.mars.internal-1719319735.446834-17225-16-15baf72b-ea4a-4221-90e5-aa04f3ba8f46.nc to Data/climate_data/NetCDF/download_2021_04_01_04.nc (1.3K)
2024-06-26 09:55:48,164 INFO Download rate 140.9K/s
2024-06-26 09:55:48,201 INFO Welcome to the CDS
2024-06-26 09:55:48,202 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-06-26 09:55:48,251 INFO Downloading https://download-0000-clone.copernicus-climate.eu/cache-compute-0000/cache/data8/adaptor.mars.internal-1719387917.2507021-1088-4-9a403d12-a5df-4b77-95f5-029fe14b670a.nc to Data/climate_data/NetCDF/download_2021_04_01_05.nc (1.3K)
2024-06-26 09:55:48,262 INFO Download rate 137.7K/s
2024-06-26 09:55:48,302 INFO Welcome to the CDS
2024-06-26 09:55:48,303 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels

...

2024-06-26 10:00:56,140 INFO Downloading https://download-0013-clone.copernicus-climate.eu/cache-compute-0013/cache/data1/adaptor.mars.internal-1719326591.336153-6102-4-5ff78fef-f6b0-4445-ad00-2461bc95cdef.nc to Data/climate_data/NetCDF/download_2021_04_30_20.nc (1.3K)
2024-06-26 10:00:56,180 INFO Download rate 33.3K/s
2024-06-26 10:00:56,218 INFO Welcome to the CDS
2024-06-26 10:00:56,219 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-06-26 10:00:56,366 INFO Downloading https://download-0018.copernicus-climate.eu/cache-compute-0018/cache/data8/adaptor.mars.internal-1719390307.8439336-19830-15-d197e1ce-afd5-4e82-a135-f82dd4a6fd0c.nc to Data/climate_data/NetCDF/download_2021_04_30_21.nc (1.3K)
2024-06-26 10:00:56,387 INFO Download rate 65K/s


<xarray.Dataset>
Dimensions:    (latitude: 1, longitude: 1, time: 1)
Coordinates:
  * longitude  (longitude) float32 15.56
  * latitude   (latitude) float32 41.54
  * time       (time) datetime64[ns] 2021-04-30T20:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-25 14:43:11 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...
done


2024-06-26 10:00:56,426 INFO Welcome to the CDS
2024-06-26 10:00:56,427 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-06-26 10:00:56,476 INFO Downloading https://download-0003-clone.copernicus-climate.eu/cache-compute-0003/cache/data5/adaptor.mars.internal-1719326595.7849014-26282-8-5daf8764-4cc7-47ca-98a8-e2cb1b5643aa.nc to Data/climate_data/NetCDF/download_2021_04_30_22.nc (1.3K)
2024-06-26 10:00:56,515 INFO Download rate 33.9K/s
2024-06-26 10:00:56,553 INFO Welcome to the CDS
2024-06-26 10:00:56,554 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-06-26 10:00:56,605 INFO Downloading https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data8/adaptor.mars.internal-1719326598.1509721-8318-9-0e2394c7-e499-4fa3-adf2-7e1eb64e1236.nc to Data/climate_data/NetCDF/download_2021_04_30_23.nc (1.3K)
2024-06-26 10:00:56,615 INFO Download rate 145.9K/s


<xarray.Dataset>
Dimensions:    (latitude: 1, longitude: 1, time: 1)
Coordinates:
  * longitude  (longitude) float32 15.56
  * latitude   (latitude) float32 41.54
  * time       (time) datetime64[ns] 2021-04-30T21:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-26 08:25:08 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...
done
<xarray.Dataset>
Dimensions:    (latitude: 1, longitude: 1, time: 1)
Coordinates:
  * longitude  (longitude) float32 15.56
  * latitude   (latitude) float32 41.54
  * time       (time) datetime64[ns] 2021-04-30T22:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-25 14:43:15 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...
done
<xarray.Dataset>
Dimensions:    (latitude: 1, longitude: 1, ti

,2m_temperature,total_precipitation,Year,Month,Day,Time,Province
0,282.087158,0.0,2021,04,01,00:00,Ravenna
1,282.087158,0.0,2021,04,01,00:00,Ravenna
2,281.537109,0.0,2021,04,01,01:00,Ravenna
3,282.087158,0.0,2021,04,01,00:00,Ravenna
4,281.537109,0.0,2021,04,01,01:00,Ravenna
...,...,...,...,...,...,...,...
17995,295.683594,0.0,2021,04,30,19:00,Foggia
17996,294.198242,0.0,2021,04,30,20:00,Foggia
17997,293.137451,0.0,2021,04,30,21:00,Foggia
17998,292.110596,0.0,2021,04,30,22:00,Foggia


#### Trasformation in daily information

Temperature dataset for getting average temperature, maximum temperature, minimum temperature

In [ ]:
temp_df = final_df.drop('total_precipitation', axis = 1)


In [ ]:
# Combina Year, Month, Day e Time in una colonna DateTime
temp_df['DateTime'] = pd.to_datetime(temp_df['Year'].astype(str) + '-' +
                                     temp_df['Month'].astype(str) + '-' +
                                     temp_df['Day'].astype(str) + ' ' +
                                     temp_df['Time'])

temp_df = temp_df[['DateTime', '2m_temperature', 'Province']]

# Funzione per calcolare Tmax, Tmin e Tavg per una singola provincia
def calculate_temperatures(temp):
    temp_h = temp.set_index('DateTime')

    # Maximum temperature
    temp_d_max = temp_h.resample('D').max().reset_index().rename(columns={"2m_temperature": "Tmax"})
    temp_d_max['Tmax'] = temp_d_max['Tmax'] - 273.15  # Convert to Celsius

    # Minimum temperature
    temp_d_min = temp_h.resample('D').min().reset_index().rename(columns={"2m_temperature": "Tmin"})
    temp_d_min['Tmin'] = temp_d_min['Tmin'] - 273.15  # Convert to Celsius

    # Average temperature
    temp_d_avg = temp_h.resample('D').mean().reset_index().rename(columns={"2m_temperature": "Tavg"})
    temp_d_avg['Tavg'] = temp_d_avg['Tavg'] - 273.15  # Convert to Celsius

    # Merge in a unique dataframe
    df2 = pd.merge(temp_d_avg, temp_d_max, on='DateTime')
    temp_d = pd.merge(df2, temp_d_min, on='DateTime')

    return temp_d[['DateTime', 'Tavg', 'Tmax', 'Tmin']]

# Applicare la funzione a tutte le province e concatenare i risultati
results = []

for province, group in temp_df.groupby('Province'):
    temp_d = calculate_temperatures(group)
    temp_d['Province'] = province
    results.append(temp_d)

# Concatenare tutti i risultati in un unico dataframe
temp_d = pd.concat(results)
temp_d = temp_d.rename(columns={'DateTime': 'Date'})  # Rinominare DateTime in Date per coerenza
temp_d = temp_d[['Date', 'Province', 'Tavg', 'Tmax', 'Tmin']]  # Riorganizzare le colonne
temp_d

,Date,Province,Tavg,Tmax,Tmin
0,2021-04-01,Foggia,14.875610,23.299469,8.558990
1,2021-04-02,Foggia,15.880920,23.860504,10.991608
2,2021-04-03,Foggia,14.929626,20.949860,11.334869
3,2021-04-04,Foggia,11.909241,14.403961,6.709381
4,2021-04-05,Foggia,9.376770,15.366608,5.557526
5,2021-04-06,Foggia,11.659851,16.971832,5.144440
6,2021-04-07,Foggia,6.859406,9.277252,5.133453
7,2021-04-08,Foggia,8.067230,13.068756,4.250153
8,2021-04-09,Foggia,8.539520,17.544098,1.040436
9,2021-04-10,Foggia,10.252411,20.027496,1.990631


Daily precipitation dataset: cumulative precipitation per day

In [ ]:
#Division of the dataset and obtaining the precipitation information
prec = final_df.drop(['2m_temperature'], axis = 1)

In [ ]:
# Combina Year, Month, Day e Time in una colonna DateTime
prec['DateTime'] = pd.to_datetime(prec['Year'].astype(str) + '-' +
                                     prec['Month'].astype(str) + '-' +
                                     prec['Day'].astype(str))


In [ ]:
#daily precipitation dataset
prec_h = prec.copy()
prec_d = prec_h.groupby(['DateTime', 'Province'])['total_precipitation'].sum().reset_index()
prec_d['DateTime'] = pd.to_datetime(prec_d['DateTime'])
prec_d = prec_d.rename(columns={'total_precipitation': 'prec'})
prec_d = prec_d.rename(columns={'DateTime': 'Date'})
prec_d

,Date,Province,prec
0,2021-04-01,Foggia,9.580325e-05
1,2021-04-01,Ravenna,0.000000e+00
2,2021-04-02,Foggia,1.176069e-02
3,2021-04-02,Ravenna,0.000000e+00
4,2021-04-03,Foggia,1.866906e-02
5,2021-04-03,Ravenna,1.511773e-02
6,2021-04-04,Foggia,3.283079e-02
7,2021-04-04,Ravenna,0.000000e+00
8,2021-04-05,Foggia,0.000000e+00
9,2021-04-05,Ravenna,0.000000e+00


Definitive dataset

In [ ]:
df_def = pd.merge(temp_d, prec_d, on=['Province', 'Date'], how='inner')

# Convertire la colonna 'Date' in datetime
df_def['Date'] = pd.to_datetime(df_def['Date'])

# Estrarre l'anno, il mese e il giorno
df_def['Year'] = df_def['Date'].dt.year
df_def['Month'] = df_def['Date'].dt.month
df_def['Day'] = df_def['Date'].dt.day

# Rimuovere la colonna 'Date' originale se non è più necessaria
df_def.drop('Date', axis=1, inplace=True)
df_def

,Province,Tavg,Tmax,Tmin,prec,Year,Month,Day
0,Foggia,14.875610,23.299469,8.558990,9.580325e-05,2021,4,1
1,Foggia,15.880920,23.860504,10.991608,1.176069e-02,2021,4,2
2,Foggia,14.929626,20.949860,11.334869,1.866906e-02,2021,4,3
3,Foggia,11.909241,14.403961,6.709381,3.283079e-02,2021,4,4
4,Foggia,9.376770,15.366608,5.557526,0.000000e+00,2021,4,5
5,Foggia,11.659851,16.971832,5.144440,4.233468e-03,2021,4,6
6,Foggia,6.859406,9.277252,5.133453,1.912410e-02,2021,4,7
7,Foggia,8.067230,13.068756,4.250153,1.298956e-05,2021,4,8
8,Foggia,8.539520,17.544098,1.040436,0.000000e+00,2021,4,9
9,Foggia,10.252411,20.027496,1.990631,0.000000e+00,2021,4,10
